## Es. 1:
Scrivere una funzione `sum n` che calcoli **ricorsivamente** la somma di tutti i numeri da 0 a n. Usarla poi per calcolare `sum 1000000`

### Soluzione:

Proviamo la prima soluzione che ci viene in mente, ovvero una funzione ricorsiva `sum n` che restituisca `n + sum (n-1)`. Per distinguere il caso base (`n = 0`) dal caso induttivo, possiamo usare un costrutto match su `n` oppure un costrutto `if n = 0 then ... else ...`
Questo modo di procedere è giusto, e il codice presentato è semanticamente corretto, ma eseguendo `sum 1000000` possiamo vedere che va in stack overflow. Infatti questo codice esegue una chiamata ricorsiva per ogni intero da `n` fino a `0`, ed evidentemente nello stack non è presente spazio sufficiente per ospitare un milione di frame. 

In [ ]:
let rec sum n = 
 match n with 
 | 0 -> 0
 | n -> n + sum (n - 1);;
 

 sum 1000000;;

Questo tipo di stack overflow è un effetto tipico della ricorsione, che non avviene usando uno stile di programmazione imperativo, con while e for. Come risolvere questo problema, e ottenere del codice funzionale equivalente a un ciclio while? Possiamo usare la **tail recurion**, ovvero uan funzione ricorsiva che "termina" con la chiamata ricorsiva. Nel codice di sopra, la funzione `sum` "termina" con una somma, nel senso che  l'espressione top level è una somma. Per avere una funzione tail recursive, dobbiamo quindi riscrivere `sum`, facendo in modo tale che l'espressione top level sia la chiamata ricorsiva.

Nel codice di `sum`, noi ottenevamo il risultato parziale (`sum (n-1)`), e poi lo combinavamo con il valore attuale (`n + sum(n-1)`). Per portare questa somma, "`n` più il risultato parziale", all'interno della chiamata ricorsiva, dobbiamo aggiungere un argomento, chiamato `tot`.
L'intuizione è che dentro `tot` conserviamo il risultato parziale, e quindi la chiamata ricorsiva passa da `n + sum(n-1)` a `sum(n-1, n + tot)`. Ovviamente, se `tot` contiene il risultato parziale, quando arriviano al caso base `n = 0` dobbiamo semplicemente restituire `tot`, perchè abbiamo finito tutte le chiamate ricorsive. La funzione così definita è tail recursive. 

In [2]:
let rec sum2 n tot = 
    match n with
            | 0 -> tot
            | n -> sum2 (n-1) (tot + n)
;;

sum2 1000000 0;;

val sum2 : int -> int -> int = <fun>


- : int = 500000500000


Se la consegna però richiede una funzione `sum: int -> int`, non possiamo semplicemente presentare una funzione con più argomenti. La soluzione standard è di chiamare la funzione scritta sopra `aux`, dandone tutti gli argomenti che ci servono, e fatto ciò il ruolo della funzione `sum` voluta dalla consegna sarà solo quello di chiamare `aux`

In [3]:
let sum2 n =
    let rec aux n tot =
        match n with
        | 0 -> tot
        | n -> aux (n-1) (tot + n)
    in aux n 0;;
    
sum2 1000000;;

val sum2 : int -> int = <fun>


- : int = 500000500000


### Takeaway:
- Delle volte è utile usare più argomenti di quelli previsti dalla specifica, e basta definire una funzione ausiliaria con gli argomenti necessari e poi chiamare quella funzione.
- Il parametro `tot` che abbiamo aggiunto è concettualmente simile a una variabile `tot` che aggiurneremmo in un ciclio while per calcolare il risultato in un linguaggio imperativo. Lì dove il paradigma imperativo modifica lo stato delle variabili fra un ciclio e l'altro, il paradigma funzionale modifica il valore degli argomenti fra una chiamata ricorsiva e l'altra. In un certo senso, OCaml ci obbliga a "esplicitare", come argomenti aggiuntivi, quali sono i side effect che useremmo in un linguaggio imperativo.

## Es. 2:
Scrivere una funzione `square_evens l` che prende una lista  `l` e restituisce una lista `l'`, contenente **solo** gli elementi pari di `l`, elevati al quadrato.

    square_evens [4; 1; 5; 4; 10] = [16; 16; 100]
 
 Scrivere due versioni di `square_evens`, una usando ricorsione e pattern matching, una usando funzioni di ordine superiore (non necessariamente `fold`).

### Soluzione:

**Usiamo ricorsione e pattern matching.** A lista vuota, restituiamo lista vuota. A lista non vuota, chiamiamo `x` il primo elemento e `xs` il resto della lista, e poi esegeuiamo un `if`. Il pattern a sinistra della freccia infatti è equivalente a una dichiarazione di variabili, abbiamo introdotto due nuovi nomi `x` e `xs`. Si può eseguire `if x mod 2 ...` solo **dentro** il secondo caso del pattern matching, non nel primo caso o fuori dal pattern matching, perchè altrimenti `x` non sarebbe definita. Il che ha senso, perchè possiamo controllare il primo elemento della lista solo con una lista non vuota, cioè solo dentro il secondo caso del pattern matching.

Con il costrutto if possiamo quindi sapere se `x` è pari o dispari. Nel primo caso, va elevato al quadrato, nel secondo caso, va tolto dalla lista. Come si modifica una lista in OCaml, se la lista è una struttura dati immutabile? Per "togliere" il primo elemento di una lista `x::xs`, basta restituire `xs`. Per "modficare" un elemento, bisogna costruire una nuova lista, passando da `x::xs` a `(x*x)::xs`. 

In [5]:
let rec square_evens l = 
    match l with 
    | [] -> []
    | x :: xs -> if (x mod 2 = 0) 
                    then x*x :: square_evens xs
                    else square_evens xs;;
                    
square_evens [4; 1; 5; 4; 10];;

val square_evens : int list -> int list = <fun>


- : int list = [16; 16; 100]


**Usiamo funzioni di ordine superiore.** Nella documentazione https://v2.ocaml.org/api/List.html, possiamo trovare numerose funzioni già definite per le liste in OCaml, come `List.length`, `List.rev`, `List.map`, `List.filter`, `List.fold_left` etc. Usando filter, possiamo prendere solo i numeri pari dalla lista, e usando map possiamo elevare questi nuemri pari al quadrato. Sia filter che map si aspettano una funzione come argomento, e ci sono vari modi, tutti semanticamente equivalenti,  di scrivere questa funzione

In [21]:
(*Modo 1: definiamo le funzioni che vogliamo passare come argomento usando let*)
let square_evens2 l = 
    let predicate x = (x mod 2) = 0 in
    let filtered = List.filter predicate l in
    let square x = x*x in
    let squared = List.map square filtered in
    squared;;

(*Modo 2: non definiamo le funzioni con let, ma passiamo come argomento
direttamente il letterale funzionale che ci serve, scritta usando fun.
(Un letterale funzionale è un espressione che vieve valutata a una funzione, così
come 40+2 è un letterale intero e [1;2;3] è un letterale stringa)*)
let square_evens2 l = 
    let filtered = List.filter (fun x -> x mod 2 = 0) l in
    let squared = List.map (fun x -> x*x) filtered in
    squared;;


(*Modo 3: Come il modo 2, ma anzichè definire le variabili filtered e squared,
    usiamo l'operatore |> per concatenare le funzioni (per motivi puramente estetici)
    |> è chiamato l'operatore di applicazione inversa, perchè vuole a sinistra l'argomento e 
    a destra la funzione. "x |> f |> g" è esattamente uguale a  "g(f(x))". Funziona perchè
    filter (come map) è una funzione a due argomenti, quindi "filter (fun x -> ...)" è
    una funzione a un argomento, che prende una lista e restituisce una lista.*)
let square_evens2 l = 
    l |> List.filter (fun x -> x mod 2 = 0) |> List.map (fun x -> x*x) ;;

square_evens2 [4; 1; 5; 4; 10];;


val square_evens2 : int list -> int list = <fun>


val square_evens2 : int list -> int list = <fun>


val square_evens2 : int list -> int list = <fun>


- : int list = [16; 16; 100]


**Usiamo una fold**. Usare filter e map in questo caso è la scelta più ragionevole, ma ovviamente si può (sempre) risolvere il problema usando una fold. L'idea è la stessa della soluzione ricorsiva: nella soluzione ricorsiva il risultato parziale era dato dalla chiamata ricorsiva, mentre con la fold il risultato parziale è dato dall'accumulatore. 

Usando la `fold_right`, dobbiamo decidere:
- Il tipo dell'accumulatore. Come è fatto un "risultato parziale" della nostra funzione? Visto che dobbiamo restituire una lista, il risultato parziale (cioè l'accumulatore) sarà appunto una lista di interi.
- Il valore iniziale dell'accumulatore. Questo è identico al primo caso del pattern matching della soluzione ricorsiva. In una fold infatti, se la lista `l` in input è vuota, viene restituito il valore iniziale dell'accumulatore. Come nella soluzione ricorsiva, quando `l` è vuota l'accumulatore deve essere una lista vuota.
- Il corpo della funzione `f` che passeremo a `List.fold_right`. Questa funzione ci dice come combinare un risultato parziale con il valore immediatamente precedente. Ad esempio, ci dice come combinare il terzultimo elemento con il risultato parziale che corrisponde al penultimo e ultimo elemento della lista (la fold_right parte infatti dal fondo). Questo è identico al secondo caso del pattern matching della soluzione ricorsiva. Semplicemente, nella soluzione ricorsiva il risultato parziale era `square_evens xs`, mentre nella funzione `f` il risultato parziale è l'accumulatore.

Si può usare anche la `fold_left`, ma è meno efficiente computazionalmente. La fold_left parte dal primo elemento, non dall'ultimo, quindi la funzione `f` che passeremo a `fold_left` ci dice come combinare il risultato parziale con il valore immediatamente successivo. Ad esempio, ci dice come combinare il terzo elemento con il risultato parziale dei primi due elementi. Ma il terzo elemento andrebbe aggiunto in fondo all'accumulatore, non in cima, quindi bisognerebbe scrivere `acc @ el`, che richiede tempo lineare, mentre nella `fold_right` si può scrivere `el :: acc` che richiede tempo costante.

In [22]:
(*Soluzione 3, usando fold right*)
let square_evens3 l = 
    let f el acc = if (el mod 2) = 0 
                    then el*el :: acc
                    else acc
    in
    List.fold_right f l [];;

square_evens3 [4; 1; 5; 4; 10];;

val square_evens3 : int list -> int list = <fun>


- : int list = [16; 16; 100]


## Es. 3: 

Scrivere una funzione `drop n l` che prende un intero e una lista, e scarta tutti gli elementi di `l` con **indice** multiplo di n. Usare sia la ricorsione che `fold_left` o `fold_right`.

    drop 2 [1; 2; 3; 17; 42] = [1; 3; 42]

In [23]:
(*Questo esercizio è stato visto in classe nel tutorato successivo, il 28 ottobre*)

## Es. 4:
Scrivere una funzione `ordered l` che restituisca `true` se e solo se `l` è una lista ordinata.

    ordered [1; 2; 4; 10] = true
    ordered [1; 3; 2; 10] = false

In [26]:
(*Do it yourself! Il trucco è usare un pattern matching che assegni un nome sia al primo che al secondo elemento, tipo
    match l with
    |[] -> ... 
    |x:[] -> ...
    |x::y::xs -> ...*)